In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\OneDrive\\Desktop\\Indivaual_Projects\\Chicken-Dishes-Classification-using-Fecal-image\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\OneDrive\\Desktop\\Indivaual_Projects\\Chicken-Dishes-Classification-using-Fecal-image'

In [5]:
import dagshub
dagshub.init(repo_owner='shivanshvyas29', repo_name='Chicken-Dishes-Classification-using-Fecal-image', mlflow=True)



Accessing as shivanshvyas29

Initialized MLflow to track repo "shivanshvyas29/Chicken-Dishes-Classification-using-Fecal-image"

Repository shivanshvyas29/Chicken-Dishes-Classification-using-Fecal-image initialized!

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int


In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            mlflow_uri="https://dagshub.com/shivanshvyas29/Chicken-Dishes-Classification-using-Fecal-image.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [9]:
import tensorflow as tf
import mlflow
import mlflow.keras
from urllib.parse import urlparse

c:\Users\DELL\Miniconda3\envs\cnncls\lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [10]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        # Store the configuration object which contains
        # paths, parameters, and MLflow settings
        self.config = config

    
    def _valid_generator(self):
        """
        This method creates a validation data generator.
        It is separated into its own function to keep the code modular
        and reusable.
        """

        # ImageDataGenerator arguments
        # rescale: Normalizes pixel values from [0,255] to [0,1]
        # validation_split: Reserves 30% of training data for validation
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        # Data flow arguments
        # target_size: Resizes images to match model input size
        # batch_size: Number of images processed at once
        # interpolation: Method used for resizing images
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Create ImageDataGenerator object for validation
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Load images from directory for validation
        # shuffle=False is used to keep order consistent during evaluation
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        """
        Static method to load a trained Keras model from disk.
        Static method is used because this function does not
        depend on class attributes.
        """
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        """
        This method evaluates the trained model on validation data.
        """

        # Load the trained model from the given path
        self.model = self.load_model(self.config.path_of_model)

        # Prepare validation data generator
        self._valid_generator()

        # Evaluate the model on validation data
        # Returns loss and accuracy
        self.score = self.model.evaluate(self.valid_generator)

    
    def save_score(self):
        """
        Saves the evaluation scores (loss and accuracy)
        into a JSON file for future reference.
        """

        scores = {
    "loss": float(self.score[0]),
    "accuracy": float(self.score[1])
}


        # Save scores as a JSON file
        save_json(path=Path("scores.json"), data=scores)

    

    def log_into_mlflow(self):
        """
        Logs parameters, metrics, and model into MLflow
        for experiment tracking and versioning.
        """

        # Set MLflow tracking URI
        mlflow.set_registry_uri(self.config.mlflow_uri)

        # Get tracking store type (file or database)
        tracking_url_type_store = urlparse(
            mlflow.get_tracking_uri()  #This line checks whether MLflow 
                                        #is using a local file system or a remote tracking server.
        ).scheme
        
        # Start an MLflow run
        with mlflow.start_run():

            # Log all hyperparameters
            mlflow.log_params(self.config.all_params)

            # Log evaluation metrics
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # Model Registry does not support file-based storage
            if tracking_url_type_store != "file":

                # Register the model in MLflow Model Registry
                mlflow.keras.log_model(
                    self.model,
                    "model",
                    registered_model_name="VGG16Model"
                )
            else:
                # Log model without registration
                mlflow.keras.log_model(self.model, "model")    


In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2026-01-06 14:24:31,415: INFO: common: YAML file loaded successfully: config\config.yaml]
[2026-01-06 14:24:31,421: INFO: common: YAML file loaded successfully: params.yaml]
[2026-01-06 14:24:31,423: INFO: common: Directory created at: artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 22s 3s/step - loss: 0.4266 - accuracy: 0.9569


2026/01/06 14:24:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-01-06 14:24:58,752: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2026-01-06 14:24:59,667: INFO: builder_impl: Assets written to: C:\Users\DELL\AppData\Local\Temp\tmp62ppnum_\model\data\model\assets]


c:\Users\DELL\Miniconda3\envs\cnncls\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


KeyboardInterrupt: 